## This notebook calculates the following Climate Domain Flood metrics: 
    * Avg number of flood claims per tract per year
    * Average flood insurance payout per number of claims per census tract

In [1]:
import os
import sys
import pandas as pd
import io
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import (
    append_metadata
)
from scripts.utils.file_helpers import (
    pull_csv_from_directory, upload_csv_aws
) 
pd.set_option('display.max_columns', None)

In [2]:
# read in CA census tiger file
ca_tract_county = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_tract_county = gpd.read_file(ca_tract_county)
ca_tract_county = ca_tract_county.drop(columns={'field_1', 'geometry'})
ca_tract_county.columns = ca_tract_county.columns.str.lower()
ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)

C:\Users\jesse\AppData\Local\Temp\ipykernel_18060\4083292558.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)


In [3]:
bucket_name = 'ca-climate-index'
aws_dir = '2a_subset/climate_risk/flood/loss/fema/flood_claims_ca/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

c:\Users\jesse\eagle\carb-climate-index-11\scripts\data_metric_calc\../..\scripts\utils\file_helpers.py:78: DtypeWarning: Columns (7,35,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_data)


Saved DataFrame as 'fema_claims_CA_subset.csv'


In [4]:
flood_claim_data = pd.read_csv('fema_claims_CA_subset.csv')

C:\Users\jesse\AppData\Local\Temp\ipykernel_18060\735346701.py:1: DtypeWarning: Columns (7,35,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  flood_claim_data = pd.read_csv('fema_claims_CA_subset.csv')


In [5]:
flood_claim_data.columns

Index(['agricultureStructureIndicator', 'asOfDate',
       'basementEnclosureCrawlspaceType', 'policyCount',
       'crsClassificationCode', 'dateOfLoss', 'elevatedBuildingIndicator',
       'elevationCertificateIndicator', 'elevationDifference',
       'baseFloodElevation', 'ratedFloodZone', 'houseWorship',
       'locationOfContents', 'lowestAdjacentGrade', 'lowestFloorElevation',
       'numberOfFloorsInTheInsuredBuilding', 'nonProfitIndicator',
       'obstructionType', 'occupancyType', 'originalConstructionDate',
       'originalNBDate', 'amountPaidOnBuildingClaim',
       'amountPaidOnContentsClaim',
       'amountPaidOnIncreasedCostOfComplianceClaim',
       'postFIRMConstructionIndicator', 'rateMethod',
       'smallBusinessIndicatorBuilding', 'totalBuildingInsuranceCoverage',
       'totalContentsInsuranceCoverage', 'yearOfLoss',
       'primaryResidenceIndicator', 'buildingDamageAmount',
       'buildingDeductibleCode', 'netBuildingPaymentAmount',
       'buildingPropertyValu

In [6]:
flood_claim_data.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2022-11-01T17:29:25.000Z,1.0,1,7.0,2010-01-20T00:00:00.000Z,0,NaN,NaN,NaN,X,0,2.0,NaN,NaN,1.0,0,NaN,1.0,1955-07-01T00:00:00.000Z,1997-12-18T00:00:00.000Z,708.99,0.00,0.0,0,7,0,250000.0,100000.0,2010,1,1708.0,1,708.99,87936.0,4.0,N,NaN,1,0.00,NaN,NaN,NaN,248.0,NaN,0.0,0,Winter storm,30000.0,0.0,60001,60001.0,ALAMEDA COUNTY*,20.0,NaN,1.0,87936.0,NaN,R,0,3.0,X,1.0,0,CA,Currently Unavailable,94546.0,6001.0,6.001431e+09,6.001431e+10,37.7,-122.1,7c7f6e5d-3745-4c58-9787-554e0050e062
1,0,2023-06-12T23:03:51.000Z,NaN,1,7.0,2023-01-16T00:00:00.000Z,1,NaN,NaN,NaN,AO,0,NaN,NaN,NaN,1.0,0,50.0,11.0,1950-09-12T00:00:00.000Z,2018-09-20T00:00:00.000Z,0.00,0.00,0.0,0,RatingEngine,0,250000.0,0.0,2023,0,0.0,5,0.00,64150.0,4.0,N,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0,California Atmospheric River,30000.0,0.0,60001,60001.0,ALAMEDA COUNTY*,20.0,20.0,1.0,164381.0,NaN,A,0,NaN,AO,1.0,1,CA,Currently Unavailable,94546.0,6001.0,6.001431e+09,6.001431e+10,37.7,-122.1,2ee7a5e0-4f81-44bc-b71a-4123af697a1e
2,0,2023-05-06T14:36:00.000Z,4.0,1,8.0,2023-01-12T00:00:00.000Z,0,NaN,NaN,NaN,X,0,3.0,NaN,NaN,2.0,0,NaN,1.0,1949-02-01T00:00:00.000Z,2019-03-20T00:00:00.000Z,23864.63,0.00,0.0,0,R,0,250000.0,100000.0,2023,1,22550.0,F,23864.63,110255.0,4.0,N,0.0,F,0.00,0.0,0.0,NaN,NaN,NaN,NaN,0,California Atmospheric River,30000.0,0.0,60002,60002.0,"ALAMEDA, CITY OF",97.0,NaN,0.0,137238.0,0.0,R,0,0.0,AE,1.0,0,CA,Currently Unavailable,94502.0,6001.0,6.001428e+09,6.001428e+10,37.7,-122.2,79535cd3-2b47-42c8-9a93-441951491de7
3,0,2023-08-17T15:53:49.000Z,0.0,1,8.0,2022-12-31T00:00:00.000Z,1,NaN,NaN,NaN,AE,0,3.0,NaN,NaN,1.0,0,50.0,12.0,1950-06-01T00:00:00.000Z,2017-08-29T00:00:00.000Z,20731.30,5440.48,0.0,0,RatingEngine,0,250000.0,100000.0,2022,1,9996.0,2,20731.30,374257.0,2.0,N,6440.0,1,5440.48,101337.0,0.0,NaN,NaN,NaN,NaN,0,California Atmospheric River,30000.0,0.0,60002,60002.0,"ALAMEDA, CITY OF",NaN,NaN,2.0,469254.0,109863.0,A,0,0.0,AE,1.0,0,CA,Currently Unavailable,94501.0,6001.0,6.001428e+09,6.001428e+10,37.8,-122.2,07dd66de-b216-4721-b4ab-742a179b06e4
4,0,2020-01-22T16:55:53.000Z,NaN,1,NaN,2018-04-11T00:00:00.000Z,0,1.0,NaN,NaN,X,0,3.0,NaN,NaN,1.0,0,NaN,1.0,1951-01-01T00:00:00.000Z,2015-11-14T00:00:00.000Z,NaN,NaN,NaN,0,7,0,250000.0,100000.0,2018,1,NaN,F,0.00,NaN,4.0,N,791.0,F,0.00,80000.0,0.0,NaN,NaN,NaN,0.0,0,NaN,30000.0,0.0,60002,60002.0,"ALAMEDA, CITY OF",1.0,20.0,1.0,NaN,0.0,A,0,NaN,X,1.0,0,CA,Currently Unavailable,94502.0,6001.0,6.001428e+09,6.001428e+10,37.7,-122.2,3e620912-b947-43af-ae12-a327f888

In [7]:
# select relevant columns to our metrics
columns_keep = [
    'id',
    'countyCode',
    'censusTract',
    'policyCount',
    'yearOfLoss',
    'netIccPaymentAmount',
    'netContentsPaymentAmount',
    'netBuildingPaymentAmount'
]
flood_claim_columns = flood_claim_data[columns_keep]

# adjust county and tract columns, changing name, data type, and data formatting
flood_claim_columns['countyCode'] = flood_claim_columns['countyCode'].astype(str).str[1:].str.split('.').str[0]
flood_claim_columns = flood_claim_columns.rename(columns={'countyCode':'countyfp'})

flood_claim_columns['censusTract'] = flood_claim_columns['censusTract'].apply(lambda x: '0' + str(int(float(x))) if pd.notnull(x) else x)
flood_claim_columns = flood_claim_columns.rename(columns={'censusTract':'tract'})

# drop duplicates based on event id columns, there are no duplicates
selected_columns = ['id']
flood_claim_drop_duplicates = flood_claim_columns.duplicated(subset=selected_columns, keep='first')
duplicate_count = flood_claim_drop_duplicates.sum()
duplicate_count

duplicate_count

C:\Users\jesse\AppData\Local\Temp\ipykernel_18060\844419853.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flood_claim_columns['countyCode'] = flood_claim_columns['countyCode'].astype(str).str[1:].str.split('.').str[0]


0

In [8]:
flood_claim_columns

,id,countyfp,tract,policyCount,yearOfLoss,netIccPaymentAmount,netContentsPaymentAmount,netBuildingPaymentAmount
0,7c7f6e5d-3745-4c58-9787-554e0050e062,001,06001430700,1,2010,0.0,0.00,708.99
1,2ee7a5e0-4f81-44bc-b71a-4123af697a1e,001,06001430900,1,2023,0.0,0.00,0.00
2,79535cd3-2b47-42c8-9a93-441951491de7,001,06001428301,1,2023,0.0,0.00,23864.63
3,07dd66de-b216-4721-b4ab-742a179b06e4,001,06001428100,1,2022,0.0,5440.48,20731.30
4,3e620912-b947-43af-ae12-a327f88826d1,001,06001428301,1,2018,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...
52501,2f5286c4-ccaf-4bac-a4e8-ac1f5cb0ddd7,an,NaN,1,1984,0.0,415.00,2534.25
52502,2065e8be-f40d-42d8-80cd-7d3d0e4fa6a5,an,NaN,1,1979,0.0,0.00,0.00
52503,248b7245-e7ab-4f9a-9370-b3f948aba4ec,an,NaN,1,1980,0.0,3754.50,18585.91
52504,48df698a-0e91-4d69-ac93-d31dc094999e,an,NaN,1,1983,0.0,10000.00,14231.17


In [9]:
# drop rows that do not have location identifier
flood_claim_cleaned = flood_claim_columns.dropna(subset=['countyfp', 'tract'], how='all')


In [10]:
flood_claim_columns.policyCount.unique()

array([  1,  10,  39,   5,   4,  15,   9,   6,  21,  25,  32,   2,   8,
        11,  12,  14,  20,  60,  16,  52,  18,  45,  51,  24,  26,   7,
        80,  36,  40, 103,  42,   3,  62,  19], dtype=int64)

## Metric 1: Avg number of flood claims per tract per year

In [11]:
flood_claim_cleaned.head(5)

,id,countyfp,tract,policyCount,yearOfLoss,netIccPaymentAmount,netContentsPaymentAmount,netBuildingPaymentAmount
0,7c7f6e5d-3745-4c58-9787-554e0050e062,001,06001430700,1,2010,0.0,0.00,708.99
1,2ee7a5e0-4f81-44bc-b71a-4123af697a1e,001,06001430900,1,2023,0.0,0.00,0.00
2,79535cd3-2b47-42c8-9a93-441951491de7,001,06001428301,1,2023,0.0,0.00,23864.63
3,07dd66de-b216-4721-b4ab-742a179b06e4,001,06001428100,1,2022,0.0,5440.48,20731.30
4,3e620912-b947-43af-ae12-a327f88826d1,001,06001428301,1,2018,0.0,0.00,0.00


In [12]:
# group the data by tract and year, sum the results, and rename the columns
flood_claim_grouped = flood_claim_cleaned.groupby(['tract', 'yearOfLoss']).agg({
    'id': 'count',                  # Count the number of rows (entries)
    'policyCount': 'sum'            # Sum the policyCount
}).reset_index()

# Rename columns for clarity
flood_claim_grouped = flood_claim_grouped.rename(columns={'id': 'rowCount', 'policyCount': 'totalPolicyCount'})

# Display the resulting DataFrame
flood_claim_grouped.head(5)

,tract,yearOfLoss,rowCount,totalPolicyCount
0,013067030312,1996,1,1
1,020117060510,1997,1,1
2,022003950200,1979,1,1
3,022051020505,1995,1,1
4,022071001735,1995,1,1


In [13]:
# Group by tract again and calculate the mean for totalPolicyCount and rowCount
tract_averages = flood_claim_grouped.groupby('tract').agg({
    'totalPolicyCount': 'mean',     # Calculate the average totalPolicyCount
    'rowCount': 'mean'              # Calculate the average rowCount
}).reset_index()

# Rename columns for clarity
tract_averages = tract_averages.rename(columns={
    'totalPolicyCount': 'avgTotalPolicyCount',
    'rowCount': 'avgRowCount'
})

# Display the resulting DataFrame
tract_averages.head()

,tract,avgTotalPolicyCount,avgRowCount
0,013067030312,1.0,1.0
1,020117060510,1.0,1.0
2,022003950200,1.0,1.0
3,022051020505,1.0,1.0
4,022071001735,1.0,1.0


In [14]:
# merge the data with California census tract
merged_flood_claims = pd.merge(ca_tract_county, tract_averages, on='tract', how='left')
print(len(merged_flood_claims))
merged_flood_claims.tail(5)

9129


,tract,countyfp,county,avgTotalPolicyCount,avgRowCount
9124,06059001303,059,orange,1.0,1.0
9125,06059001304,059,orange,NaN,NaN
9126,06059001401,059,orange,NaN,NaN
9127,06013367200,013,contra costa,1.0,1.0
9128,06037578100,037,los angeles,NaN,NaN


In [15]:
#merged_flood_claims.avgTotalPolicyCount.max()

In [16]:
# visualize the df's before and after grouping to fact check sums and averages are correct
checking = flood_claim_cleaned[flood_claim_cleaned['tract'] == '06013360200']
print(checking)
checking = flood_claim_grouped[flood_claim_grouped['tract'] == '06013360200']
print(checking)
checking = tract_averages[tract_averages['tract'] == '06013360200']
print(checking)

                                         id countyfp        tract  \
25     69b13959-337a-4547-b6d5-618900c7acc2      013  06013360200   
39     491479cd-c597-45c9-93c4-185547bce574      013  06013360200   
1218   733df6b2-446a-4e38-aa01-862eb23aea20      013  06013360200   
1219   791bee0e-32e8-46a6-91f2-8ce4feddf7e8      013  06013360200   
1236   09a4ecde-b2ae-4fe0-be18-796c1302f717      013  06013360200   
6538   89bab78a-cfc2-4b80-b9fc-46ddcfc52408      013  06013360200   
6539   f4a637fb-23e0-4848-90f5-1c532a33d6a3      013  06013360200   
7739   38879ee6-de28-4002-af5c-fa48a68016aa      013  06013360200   
19659  998c0eb0-14a2-49a4-b652-b69ed95d0751      013  06013360200   
20871  fb23ecda-2c18-4033-91b3-69ec148094af      013  06013360200   
20904  ff6cb48b-fa6d-40fb-9549-97d122eee91a      013  06013360200   
27385  aeaf11a9-d4fe-4ab6-a2af-0dabfd058b2a      013  06013360200   
27394  501e7df6-22ed-4a00-b237-bc5ccb5e99be      013  06013360200   
27426  44c3c18b-53ad-4bfc-9ebe-b87

In [17]:
merged_flood_claims = merged_flood_claims.drop(columns={'avgRowCount'})

In [18]:
merged_flood_claims.to_csv('climate_flood_claims_metric.csv', index=False)

## Metric 2: Average flood insurance payout per number of claims per census tract

In [19]:
flood_claim_cleaned.head(5)

,id,countyfp,tract,policyCount,yearOfLoss,netIccPaymentAmount,netContentsPaymentAmount,netBuildingPaymentAmount
0,7c7f6e5d-3745-4c58-9787-554e0050e062,001,06001430700,1,2010,0.0,0.00,708.99
1,2ee7a5e0-4f81-44bc-b71a-4123af697a1e,001,06001430900,1,2023,0.0,0.00,0.00
2,79535cd3-2b47-42c8-9a93-441951491de7,001,06001428301,1,2023,0.0,0.00,23864.63
3,07dd66de-b216-4721-b4ab-742a179b06e4,001,06001428100,1,2022,0.0,5440.48,20731.30
4,3e620912-b947-43af-ae12-a327f88826d1,001,06001428301,1,2018,0.0,0.00,0.00


In [20]:
# create a new column summing relevant columns representing how much insurance paid out on these claims
flood_claim_total_cost = flood_claim_cleaned
flood_claim_total_cost['total_insurance_payment'] = flood_claim_cleaned['netIccPaymentAmount'] + flood_claim_cleaned['netContentsPaymentAmount'] + flood_claim_cleaned['netBuildingPaymentAmount']
flood_claim_total_cost

,id,countyfp,tract,policyCount,yearOfLoss,netIccPaymentAmount,netContentsPaymentAmount,netBuildingPaymentAmount,total_insurance_payment
0,7c7f6e5d-3745-4c58-9787-554e0050e062,001,06001430700,1,2010,0.0,0.00,708.99,708.99
1,2ee7a5e0-4f81-44bc-b71a-4123af697a1e,001,06001430900,1,2023,0.0,0.00,0.00,0.00
2,79535cd3-2b47-42c8-9a93-441951491de7,001,06001428301,1,2023,0.0,0.00,23864.63,23864.63
3,07dd66de-b216-4721-b4ab-742a179b06e4,001,06001428100,1,2022,0.0,5440.48,20731.30,26171.78
4,3e620912-b947-43af-ae12-a327f88826d1,001,06001428301,1,2018,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
52501,2f5286c4-ccaf-4bac-a4e8-ac1f5cb0ddd7,an,NaN,1,1984,0.0,415.00,2534.25,2949.25
52502,2065e8be-f40d-42d8-80cd-7d3d0e4fa6a5,an,NaN,1,1979,0.0,0.00,0.00,0.00
52503,248b7245-e7ab-4f9a-9370-b3f948aba4ec,an,NaN,1,1980,0.0,3754.50,18585.91,22340.41
52504,48df698a-0e91-4d69-ac93-d31dc094999e,an,NaN,1,1983,0.0,10000.00,14231.17,24231.17


In [21]:
# group the data my tract and year of loss and sum the total cost
flood_claim_cost_grouped = flood_claim_total_cost.groupby(['tract', 'yearOfLoss']).agg({
    'total_insurance_payment': 'sum'            # Sum the policyCount
}).reset_index()

# Display the resulting DataFrame
flood_claim_cost_grouped.head(5)

,tract,yearOfLoss,total_insurance_payment
0,013067030312,1996,0.00
1,020117060510,1997,87535.57
2,022003950200,1979,3366.06
3,022051020505,1995,23181.01
4,022071001735,1995,7213.73


In [22]:
# Group by countyCode and calculate the mean for total_insurance_payment
county_cost_averages = flood_claim_cost_grouped.groupby('tract').agg({
    'total_insurance_payment': 'mean'   # Calculate the average rowCount
}).reset_index()

# Rename columns for clarity
county_cost_averages = county_cost_averages.rename(columns={
    'total_insurance_payment': 'average_insurance_payout'
})

# Display the resulting DataFrame
county_cost_averages.head()

,tract,average_insurance_payout
0,013067030312,0.00
1,020117060510,87535.57
2,022003950200,3366.06
3,022051020505,23181.01
4,022071001735,7213.73


In [23]:
# merge with CA census tracts
merged_flood_claims_cost = pd.merge(ca_tract_county, county_cost_averages, on='tract', how='left')
print(len(merged_flood_claims_cost))
merged_flood_claims_cost.tail()

9129


,tract,countyfp,county,average_insurance_payout
9124,06059001303,059,orange,0.00
9125,06059001304,059,orange,NaN
9126,06059001401,059,orange,NaN
9127,06013367200,013,contra costa,703.89
9128,06037578100,037,los angeles,NaN


In [24]:
# visualize the results before and after merge steps to fact check
checking = flood_claim_cost_grouped[flood_claim_cost_grouped['tract'] == '06013360200']
print(checking)
checking = county_cost_averages[county_cost_averages['tract'] == '06013360200']
print(checking)
checking = merged_flood_claims_cost[merged_flood_claims_cost['tract'] == '06013360200']
print(checking)

            tract  yearOfLoss  total_insurance_payment
1127  06013360200        1983                   622.31
1128  06013360200        1986                 23429.72
1129  06013360200        1993                  6861.75
1130  06013360200        1995                  5579.34
1131  06013360200        1998                 34561.83
1132  06013360200        2003                     0.00
1133  06013360200        2011                     0.00
1134  06013360200        2017                482871.83
1135  06013360200        2021                171451.70
1136  06013360200        2023                133861.90
           tract  average_insurance_payout
343  06013360200                 85924.038
            tract countyfp        county  average_insurance_payout
9093  06013360200      013  contra costa                 85924.038


In [25]:
# merge the new df again to our first metric df so we can calculate cost per policy per tract
cost_claims_merge = pd.merge(merged_flood_claims_cost, merged_flood_claims, on='tract', how='left')
cost_claims_merge = cost_claims_merge.drop(columns={'countyfp_x','county_x', 'countyfp_y', 'county_y'})

In [26]:
cost_claims_merge

,tract,average_insurance_payout,avgTotalPolicyCount
0,06085504321,0.00,2.0
1,06085504410,0.00,1.0
2,06085507003,NaN,NaN
3,06085507004,NaN,NaN
4,06085502204,NaN,NaN
...,...,...,...
9124,06059001303,0.00,1.0
9125,06059001304,NaN,NaN
9126,06059001401,NaN,NaN
9127,06013367200,703.89,1.0


In [27]:
# perform the calculation on avg total policy county
cost_claims_merge['avg_insurance_payout_per_policy'] = cost_claims_merge.apply(lambda row: row['average_insurance_payout'] / row['avgTotalPolicyCount'] 
                            if row['average_insurance_payout'] > row['avgTotalPolicyCount'] 
                            else None, axis=1)

In [28]:
print(len(cost_claims_merge.avgTotalPolicyCount.unique()))

311


In [29]:
non_nan_count = cost_claims_merge['avg_insurance_payout_per_policy'].count()
non_nan_count

2707

In [30]:
cost_claims_merge.to_csv('climate_flood_cost_metric.csv', index=False)

Function Call

In [31]:
@append_metadata
def flood_claims_cost_upload(input_csv, export=False, varname=''):
    '''
    Uploads the flood claim and cost metrics to S3 bucket. The metrics are:
    
    * Avg number of flood claims per tract per year
    * Average flood insurance payout per number of claims per census tract
    
    Data for these metrics are sourced from FEMA's redacted NFIP claims at:
    https://www.google.com/url?q=https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2&sa=D&source=editors&ust=1723749642983941&usg=AOvVaw0-Ri52Pad7wzLYu2eNKABx

    Methods
    -------
    Relevant columns to our data metrics were isolated, renamed, and adjusted for consistency.
    Data was isolated to include data in non-nan census tracts.
    Duplicates were removed based on event ID.
    Data were grouped by tract and year then summed to identify number of events/cost per tract per year.
    Data were grouped again by tract and averaged to identify average cost/number of policies per census tract.
    For the insurance payout metric, the average values were divided my our first
    Parameters
    ----------
    input_csv: string
        csv flood claim/cost data 
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI flood claim/cost  metrics to AWS
        True = will upload resulting df containing CAL CRAI flood claim/cost metrics to AWS

    Script
    ------
    climate_flood_claims.ipynb

    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    '''
    print('Data transformation: relevant columns were isolated and renamed')
    print('Data transformation: duplicate entries by event ID were dropped.')
    print('Data transformation: data was grouped by tract & year then summed, then grouped once again and averaged.')
    print('Data transformation: data was merged with California census tracts.') 
 
    if export == True:
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [input_csv]
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{input_csv} uploaded to AWS.')
 
    if os.path.exists(input_csv):
        os.remove(input_csv)

In [32]:
input_csvs = ['climate_flood_claims_metric.csv',
               'climate_flood_cost_metric.csv']

varnames = ['climate_fema_nfip_claims',
            'climate_fema_nfip_claim_cost']

# Process the data and export
for input_csv, varname in zip(input_csvs, varnames):
    print(f'Processing {input_csv} with varname {varname}')
    flood_claims_cost_upload(input_csv, export=True, varname='test')
    print(f'Completed uploading {input_csv} with varname {varname}')

Processing climate_flood_claims_metric.csv with varname climate_fema_nfip_claims
Completed uploading climate_flood_claims_metric.csv with varname climate_fema_nfip_claims
Processing climate_flood_cost_metric.csv with varname climate_fema_nfip_claim_cost
Completed uploading climate_flood_cost_metric.csv with varname climate_fema_nfip_claim_cost
